In [ ]:
!pip install noisereduce
!pip install python_speech_features
!pip install pandas
!pip install tensorflow
!pip install SpeechRecognition
!pip install python_speech_features
!pip install --upgrade librosa

In [ ]:
import os
import glob
from tqdm import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.io import wavfile
from python_speech_features import mfcc , logfbank
import librosa as lr
import os, glob, pickle
import librosa
from scipy import signal
import noisereduce as nr
from glob import glob
import librosa
get_ipython().magic('matplotlib inline')
import soundfile
import tensorflow
from tensorflow.keras.layers import Conv2D,MaxPool2D, Flatten, LSTM
from keras.layers import Dropout,Dense,TimeDistributed
from keras.models import Sequential
from keras.utils import to_categorical 
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np

In [ ]:
dirName = 'C:/Users/Admin/Desktop/RAW'
dir_path = 'C:/Users/Admin/Desktop/RAW'

In [ ]:
list_of_files = []
for (dirpath, dirnames, filenames) in os.walk(dir_path):
    for file in filenames:
        if file.endswith('.wav'):
            list_of_files.append(os.path.join(dirpath, file))

num_audio_files = len(list_of_files)
print(f'Total number of audio files: {num_audio_files}')

In [ ]:
def getListOfFiles(dirName):
    listOfFile=os.listdir(dirName)
    allFiles=list()
    for entry in listOfFile:
        fullPath=os.path.join(dirName, entry)
        if os.path.isdir(fullPath):
            allFiles=allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
    return allFiles

dirName = 'C:/Users/Admin/Desktop/RAW'
listOfFiles = getListOfFiles(dirName)
len(listOfFiles)

In [ ]:
import speech_recognition as sr
r=sr.Recognizer()
for file in range(0 , len(listOfFiles) , 1):
    with sr.AudioFile(listOfFiles[file]) as source:
        audio = r.listen(source)
        try:
            text = r.recognize_google(audio)
            print(text)
        except:
            print('error')

In [ ]:
def envelope(y , rate, threshold):
    mask=[]
    y=pd.Series(y).apply(np.abs)
    y_mean = y.rolling(window=int(rate/10) ,  min_periods=1 , center = True).mean()
    for mean in y_mean:
        if mean>threshold:
            mask.append(True)
        else:
            mask.append(False)
    return mask

In [ ]:
for file in range(0 , len(listOfFiles) , 1):
    audio , sfreq = lr.load(listOfFiles[file])
    time = np.arange(0 , len(audio)) / sfreq
    
    fig ,ax = plt.subplots()
    ax.plot(time , audio)
    ax.set(xlabel = 'Time (s)' , ylabel = 'Sound Amplitude')
    plt.show()

for file in range(0 , len(listOfFiles) , 1):
     sample_rate , samples = wavfile.read(listOfFiles[file])
     frequencies , times, spectrogram = signal.spectrogram(samples, sample_rate) 
     plt.pcolormesh(times, frequencies, spectrogram)
     plt.imshow(spectrogram)
     plt.ylabel('Frequency [Hz]')
     plt.xlabel('Time [sec]')
     plt.show()
     #14min

In [ ]:
print(samples.dtype)
print(spectrogram.shape)
print(spectrogram.dtype)
#spectrogram has 129 frequency bins and 779 time slices#
plt.pcolormesh(times, frequencies, spectrogram)

In [ ]:
#for file in range(len(listOfFiles)):
#    sample_rate, samples = wavfile.read(listOfFiles[file])
#    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)
#
#    plt.pcolormesh(times, frequencies, spectrogram)
#    plt.ylabel('Frequency [Hz]')
#    plt.xlabel('Time [sec]')
#    plt.show()


In [ ]:
spectrogram = np.transpose(spectrogram)
times = times.reshape(-1, 1)
frequencies = frequencies.reshape(1, -1)

In [ ]:
for file in range(len(listOfFiles)):
    sample_rate, samples = wavfile.read(listOfFiles[file])
    frequencies, times, spectrogram = signal.spectrogram(samples, sample_rate)

    plt.pcolormesh(times, frequencies, spectrogram)
    plt.ylabel('Frequency [Hz]')
    plt.xlabel('Time [sec]')
    plt.show()
#6min

In [ ]:
def plot_signals(signals):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Time Series' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(signals.keys())[i])
            axes[x,y].plot(list(signals.values())[i])
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def plot_fft(fft):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Fourier Transform' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            data = list(fft.values())[i]
            Y,freq = data[0] , data[1]
            axes[x,y].set_title(list(fft.keys())[i])
            axes[x,y].plot(freq , Y)
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1
    
def plot_fbank(fbank):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Filter Bank Coefficients' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(fbank.keys())[i])
            axes[x,y].imshow(list(fbank.values())[i],cmap='hot', interpolation = 'nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1
            
def plot_mfccs(mfccs):
    fig , axes = plt.subplots(nrows=2, ncols=5,sharex =False , sharey=True, figsize=(20,5))
    fig.suptitle('Mel Frequency Capstrum  Coefficients' , size=16)
    i=0
    for x in range(2):
        for y in range(5):
            axes[x,y].set_title(list(mfccs.keys())[i])
            axes[x,y].imshow(list(mfccs.values())[i],
                             cmap='hot', interpolation = 'nearest')
            axes[x,y].get_xaxis().set_visible(False)
            axes[x,y].get_yaxis().set_visible(False)
            i +=1

def calc_fft(y,rate):
    n = len(y)
    freq = np.fft.rfftfreq(n , d= 1/rate)
    Y= abs(np.fft.rfft(y)/n)
    return(Y,freq)

In [ ]:
for file in tqdm(glob.glob('C:\\Users\\Admin\\Desktop\\RAW\\**\**\*.wav', recursive=True)):
    file_name = os.path.basename(file)
    signal, rate = librosa.load(file, sr=16000)
    mask = envelope(signal, rate, 0.0005)
    wavfile.write(filename='C:\\Users\\Admin\\Desktop\\CLEAN\\' + str(file_name), rate=rate, data=signal[mask])


In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = None
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            mfccs = mfccs.reshape(1, -1)  # Reshape to have a single row
            result = mfccs
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            if result is None:
                result = chroma.reshape(1, -1)
            else:
                chroma = chroma.reshape(1, -1)
                result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            if result is None:
                result = mel.reshape(1, -1)
            else:
                mel = mel.reshape(1, -1)
                result = np.hstack((result, mel))
    return result

file_path = "C:\\Users\\Admin\\Desktop\\CLEAN\\03-01-01-01-01-01-02.wav"
mfcc = True
chroma = True
mel = True

features = extract_feature(file_path, mfcc, chroma, mel)
print(features)


In [ ]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
file_path = "C:\\Users\\Admin\\Desktop\\CLEAN\\03-01-01-01-01-01-02.wav"
mfcc = True
chroma = True
mel = True
features = extract_feature(file_path, mfcc, chroma, mel)
print(features)

In [ ]:
def load_data(path, test_size=0.33):
    x, y = [], []
    for file in glob.glob(os.path.join(path, '*.wav')):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append([emotion, file_name])
        print("Number of samples:", len(x))
        print("Number of labels:", len(y))
        return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
def load_data(test_size=0.33):
    x,y=[],[]
    answer = 0
    for file in glob.glob(r'C:\\Users\\Admin\\Desktop\\CLEAN\\*.wav'):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            answer += 1
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append([emotion,file_name])
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
import glob

x_train, x_test, y_train, y_test = load_data(test_size=0.25)
print(np.shape(x_train), np.shape(x_test), np.shape(y_train), np.shape(y_test))

print('Sample feature:', x_train[0])
print('Test ','Sample label:', y_train[0])

y_test_map = np.array(y_test).T
y_test = y_test_map[0]
test_filename = y_test_map[1]
y_train_map = np.array(y_train).T
y_train = y_train_map[0]
train_filename = y_train_map[1]
print(np.shape(y_train), np.shape(y_test))
print(*test_filename, sep="\n")


In [ ]:
print((x_train[0], x_test[0]))
print(f'Features extracted: {x_train.shape[1]}')

In [ ]:
from sklearn.neural_network import MLPClassifier

x_train = x_train.reshape(x_train.shape[0], -1)
y_train = y_train.reshape(y_train.shape[0],)

model = MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)
model.fit(x_train, y_train)


In [ ]:
Pkl_Filename = "Emotion_Voice_Detection_Model.pkl"  

with open(Pkl_Filename, 'wb') as file:  
    pickle.dump(model, file)

In [ ]:
# Load the Model back from file
with open(Pkl_Filename, 'rb') as file:  
    Emotion_Voice_Detection_Model = pickle.load(file)

Emotion_Voice_Detection_Model

In [ ]:
# Reshape x_test
num_samples, num_features, num_time_steps = x_test.shape
x_test_reshaped = x_test.reshape(num_samples, num_features * num_time_steps)

# Make predictions on the reshaped data
y_pred = model.predict(x_test_reshaped)
print(y_pred)


In [ ]:
#Store the Prediction probabilities into CSV file 
import numpy as np
import pandas as pd
y_pred1 = pd.DataFrame(y_pred, columns=['predictions'])
y_pred1['file_names'] = test_filename
print(y_pred1)
y_pred1.to_csv('predictionfinal.csv')

Record audio using audio_record

In [ ]:
import librosa
import librosa.display
import matplotlib.pyplot as plt

# Load audio file
file_path = 'output10.wav'
data, sampling_rate = librosa.load(file_path)

# Extract features
mfccs = librosa.feature.mfcc(y=data, sr=sampling_rate)
chroma = librosa.feature.chroma_stft(y=data, sr=sampling_rate)
mel = librosa.feature.melspectrogram(y=data, sr=sampling_rate)

# Plot features
plt.figure(figsize=(12, 6))

# MFCC
plt.subplot(3, 1, 1)
librosa.display.specshow(mfccs, x_axis='time')
plt.colorbar(format='%+2.0f dB')
plt.title('MFCC')
plt.xlabel('Time')
plt.ylabel('MFCC Coefficients')

# Chroma
plt.subplot(3, 1, 2)
librosa.display.specshow(chroma, x_axis='time')
plt.colorbar()
plt.title('Chroma')
plt.xlabel('Time')
plt.ylabel('Pitch Class')

# Mel spectrogram
plt.subplot(3, 1, 3)
librosa.display.specshow(librosa.power_to_db(mel, ref=np.max), x_axis='time', y_axis='mel')
plt.colorbar(format='%+2.0f dB')
plt.title('Mel Spectrogram')
plt.xlabel('Time')
plt.ylabel('Frequency')

plt.tight_layout()
plt.show()
# Plot waveform
plt.figure(figsize=(15, 5))
plt.plot(data)
plt.title('Waveform')
plt.xlabel('Time')
plt.ylabel('Amplitude')
plt.show()


In [ ]:
import librosa
import soundfile
import numpy as np

file = "C:\\Users\\Admin\\Desktop\\TEST Audio\\Test 2.wav"

ans = []
new_feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
ans.append(new_feature)
ans = np.array(ans)

# Reshape the ans array to have 2 dimensions
ans = np.reshape(ans, (ans.shape[0], -1))

prediction = Emotion_Voice_Detection_Model.predict(ans)
print(prediction)


REDIRECT THE OUTPUT TO music_recommendation 